In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from scipy.optimize import curve_fit
from tqdm import tqdm

import warnings
warnings.filterwarnings("ignore")
import os
from sklearn.model_selection import GridSearchCV

In [ ]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 MB 10.1 MB/s eta 0:00:00


In [ ]:
from catboost import CatBoostRegressor

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving train.csv to train.csv


In [ ]:
df = pd.read_csv('train.csv', parse_dates=['datetime'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67136 entries, 0 to 67135
Data columns (total 20 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   datetime                     67136 non-null  datetime64[ns]
 1   Номер скважины               67136 non-null  int64         
 2   Дебит нефти                  67136 non-null  float64       
 3   Давление забойное            64756 non-null  float64       
 4   x                            67136 non-null  float64       
 5   y                            67136 non-null  float64       
 6   Объем жидкости               3465 non-null   float64       
 7   Объем нефти                  3465 non-null   float64       
 8   Активная мощность (ТМ)       62412 non-null  float64       
 9   Время работы (ТМ)            63475 non-null  float64       
 10  Газовый фактор рабочий (ТМ)  25230 non-null  float64       
 11  Давление буферное            10208 non-nu

Посмотрим на имеющийся датасет:


In [ ]:
df.head(10)

,datetime,Номер скважины,Дебит нефти,Давление забойное,x,y,Объем жидкости,Объем нефти,Активная мощность (ТМ),Время работы (ТМ),Газовый фактор рабочий (ТМ),Давление буферное,Давление забойное от Hд,Давление забойное от Pпр,Давление линейное (ТМ),Давление на входе ЭЦН (ТМ),Дебит газа (ТМ),Дебит газа попутного,Дебит жидкости (ТМ),Коэффициент мощности (ТМ)
0,1990-08-01,0,19.939,NaN,18670.86,5714.86,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1990-08-02,0,19.939,39.064454,18670.86,5714.86,NaN,NaN,50.851351,9.600833,NaN,NaN,33.913336,39.064454,1.887872,30.293676,3843.746667,24.25,24.250000,98.534314
2,1990-08-03,0,21.172,39.064487,18670.86,5714.86,NaN,NaN,52.353846,NaN,NaN,NaN,33.806090,39.064487,1.885714,30.261774,3900.955000,25.75,25.833333,99.139785
3,1990-08-04,0,22.529,38.965297,18670.86,5714.86,NaN,NaN,51.242424,9.600000,NaN,NaN,33.695717,38.965297,1.875851,30.212768,3874.505000,27.40,27.933333,98.744318
4,1990-08-05,0,22.529,38.766822,18670.86,5714.86,NaN,NaN,50.910256,9.563889,NaN,NaN,33.695717,38.766822,1.873163,30.109119,3853.696667,27.40,27.400000,98.419689
5,1990-08-06,0,26.558,39.064668,18670.86,5714.86,NaN,NaN,52.773333,9.536389,NaN,NaN,33.394049,39.064668,1.895426,30.187817,3604.086667,32.30,33.200000,98.827411
6,1990-08-07,0,22.150,39.367223,18670.86,5714.86,NaN,NaN,52.766234,9.500000,NaN,NaN,33.947313,39.367223,1.914947,30.396186,3336.436667,30.00,37.933333,98.981395
7,1990-08-08,0,22.888,39.367223,18670.86,5714.86,788.311,757.568,52.759036,9.600278,NaN,NaN,33.947313,39.367223,1.906082,30.361905,3051.030000,NaN,40.700000,98.847619
8,1990-08-09,0,23.626,39.268002,18670.86,5714.86,NaN,NaN,49.012821,9.649722,NaN,NaN,33.833863,39.268002,1.898506,30.348865,3438.913333,32.00,36.433333,98.254054
9,1990-08-10,0,23.626,39.367282,18670.86,5714.86,NaN,NaN,53.192308,9.600833,NaN,NaN,33.833863,39.367282,1.897708,30.395800,3554.626667,32.00,27.233333,99.130000


In [ ]:
df.corr()

,Номер скважины,Дебит нефти,Давление забойное,x,y,Объем жидкости,Объем нефти,Активная мощность (ТМ),Время работы (ТМ),Газовый фактор рабочий (ТМ),Давление буферное,Давление забойное от Hд,Давление забойное от Pпр,Давление линейное (ТМ),Давление на входе ЭЦН (ТМ),Дебит газа (ТМ),Дебит газа попутного,Дебит жидкости (ТМ),Коэффициент мощности (ТМ)
Номер скважины,1.000000,-0.167026,0.041373,-0.049166,0.044745,0.090369,-0.050871,0.065275,0.043440,-0.004552,-0.044767,0.088518,0.065820,-0.010010,0.018197,-0.002168,0.057736,0.012970,-0.107194
Дебит нефти,-0.167026,1.000000,-0.208876,0.233646,-0.364306,0.034383,0.787894,0.020460,0.007884,-0.029139,0.306256,-0.241225,-0.204389,0.020226,-0.017785,0.007697,-0.038269,-0.004439,0.023887
Давление забойное,0.041373,-0.208876,1.000000,-0.144263,0.144864,0.145133,-0.298749,0.159151,0.222273,0.009170,-0.059168,0.911375,0.897171,0.008483,-0.004189,-0.000624,0.323707,0.069526,-0.042921
x,-0.049166,0.233646,-0.144263,1.000000,-0.482897,0.014265,0.340361,-0.059699,-0.121678,-0.021206,0.162918,-0.149963,-0.180804,0.079792,0.012024,0.016904,-0.128348,-0.024914,0.002611
y,0.044745,-0.364306,0.144864,-0.482897,1.000000,0.205590,-0.440315,0.205360,0.337782,-0.025226,-0.505489,0.148237,0.164619,-0.229816,0.007114,-0.013254,0.368276,0.064852,0.074140
Объем жидкости,0.090369,0.034383,0.145133,0.014265,0.205590,1.000000,0.142911,0.731327,0.517607,-0.023768,-0.074749,0.115649,-0.023421,0.017648,-0.020052,0.126007,0.820463,0.829907,0.110504
Объем нефти,-0.050871,0.787894,-0.298749,0.340361,-0.440315,0.142911,1.000000,0.083503,0.025386,0.009601,0.235622,-0.351552,-0.321310,0.175610,-0.004536,0.119605,-0.017354,-0.018915,0.078968
Активная мощность (ТМ),0.065275,0.020460,0.159151,-0.059699,0.205360,0.731327,0.083503,1.000000,0.344648,-0.013540,0.017517,0.123150,0.046397,-0.050163,0.002563,0.007209,0.808730,0.135569,0.073235
Время работы (ТМ),0.043440,0.007884,0.222273,-0.121678,0.337782,0.517607,0.025386,0.344648,1.000000,-0.027290,-0.086639,0.201888,0.101222,-0.007818,-0.014518,0.001842,0.650091,0.121307,0.092713
Газовый фактор рабочий (ТМ),-0.004552,-0.029139,0.009170,-0.021206,-0.025226,-0.023768,0.009601,-0.013540,-0.027290,1.000000,0.046912,-0.002187,0.003374,0.071561,-0.006458,0.018806,-0.012519,-0.004316,0.036900


In [ ]:
df['Дебит нефти'].describe()

count    67136.000000
mean        10.800321
std          8.894787
min          0.003000
25%          4.639000
50%          8.811292
75%         14.126000
max         79.977000
Name: Дебит нефти, dtype: float64

Создадим массив со значениями номеров скважин, посмотрим на их количество. 

In [ ]:
wells = df['Номер скважины'].unique()
values_of_well = df['Номер скважины'].value_counts()
values_of_well

25     740
92     716
4      716
12     716
10     692
      ... 
45     620
44     620
41     620
40     620
105    620
Name: Номер скважины, Length: 106, dtype: int64

Некоторые строчки в датасете повторяются с отличием лишь в объёмах нефти и газа. Для предсказания мы будем использовать только показатели целевой переменной,так что можно удалить все дубликаты, поскольку значения целевой переменной, даты и номера скважины у них совпадают.


In [ ]:
df_drop = df.drop_duplicates(subset=['datetime', 'Номер скважины'])

После удаления дубликатов, возьмём в датасете все даты до 1992-04-11:

In [ ]:
df_new = df_drop[df_drop['datetime'] <= '1992-04-11']
df_new['datetime'].value_counts()

1990-08-01    106
1991-09-21    106
1991-09-14    106
1991-09-15    106
1991-09-16    106
             ... 
1991-02-24    106
1991-02-25    106
1991-02-26    106
1991-02-27    106
1992-04-11    106
Name: datetime, Length: 620, dtype: int64

Выделим для каждой из 106 скважин столбец значений дебета нефти, соответствующий периоду с 1990-08-01 по 1992-04-11:


In [ ]:
array_of_wells = np.empty([106,620])
for i in range(106):
  array_of_wells[i][:] = df_new[df_new['Номер скважины'] == i]['Дебит нефти'].to_numpy() 

Сгенерируем датасет, объектом которого будет выступать часть временного ряда [X,Y], где X и Y - даты, в качестве признаков будут выступать статистики дебитов нефти на этом отрезке(минимум, максимум, среднее, стандартное отклонение, значения дебита в точке Y,), а целевой переменной будет выступать значение дебита в этом ряде в точке Y+1.


In [ ]:
debit = []
max = []
min = []
mean = []
std = []
debit_end = []
#debit_post_end = []
for i in range(20):
  for j in wells:
    debit.append(array_of_wells[j][30*i+49])
    debit_end.append(array_of_wells[j][30*i+48])
    #debit_post_end.append(array_of_wells[j][30*i+47])
    max.append(np.max(array_of_wells[j][30*i+19:30*i+49]))
    min.append(np.min(array_of_wells[j][30*i+19:30*i+49]))
    mean.append(np.mean(array_of_wells[j][30*i+19:30*i+49]))
    std.append(np.std(array_of_wells[j][30*i+19:30*i+49]))

In [ ]:
df_model = pd.DataFrame()
df_model['Дебит нефти'] = debit
df_model['max'] = max
df_model['min'] = min
df_model['mean'] = mean
df_model['std'] = std
#df_model['Дебит нефти в пред конце'] = debit_post_end
df_model['Дебит нефти в конце'] = debit_end
df_model.head()

,Дебит нефти,max,min,mean,std,Дебит нефти в конце
0,24.667,27.873,19.926,24.804483,1.247731,24.667
1,21.577,25.606,18.878,21.704217,1.740750,23.402
2,34.241,36.001,31.165,33.239067,1.274132,34.241
3,33.224,35.742,27.788,32.507950,1.710625,33.224
4,10.538,13.853,8.802,10.225767,1.312956,10.538


In [ ]:
X = df_model[['max', 'min', 'mean', 'std', 'Дебит нефти в конце']]
y = df_model['Дебит нефти']

Обучаем модель на сгенерированном датасете:

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.15, random_state=42)

In [ ]:
model = CatBoostRegressor()
model.fit(X_train, y_train, eval_set=(X_test, y_test), verbose = False)
print(f"Tree count: {str(model.tree_count_)}")

Tree count: 461


In [ ]:
forecast_horizon = 90

In [ ]:
def make_prediction(debits):
  '''The function predicts the next value of a time series 
  using a model based on the previous values passed 
  '''
  max_debits = np.max(debits)
  min_debits = np.min(debits)
  mean_debits = np.mean(debits)
  std_debits = np.std(debits)
  end_debit = debits[29]


  return model.predict([max_debits, min_debits, mean_debits, std_debits, end_debit])

Заполняем датафрейм полученными предсказаниями для каждой скважины: 

In [ ]:
all_forecasts = []
for well in wells:
  time_series = array_of_wells[well][-31:-1].tolist()
  wells_predicts = []
  for i in range(forecast_horizon):
    pred = make_prediction(time_series)
    wells_predicts.append(pred)
    time_series.append(pred)
    time_series = time_series[-30:]

  date_range = pd.date_range(start='1992-04-11', freq='1D', periods=forecast_horizon)
  forecats_df = pd.DataFrame({'datetime': date_range, 'forecast': wells_predicts})
  forecats_df['Номер скважины'] = [well] * len(forecats_df)   
  all_forecasts.append(forecats_df)
all_forecasts = pd.concat(all_forecasts)  

Проверяем, что датасет имеет правильный вид:

In [ ]:
all_forecasts.head()

,datetime,forecast,Номер скважины
0,1992-04-11,12.076849,0
1,1992-04-12,10.731937,0
2,1992-04-13,9.846014,0
3,1992-04-14,9.214055,0
4,1992-04-15,8.666457,0


In [ ]:
all_forecasts.shape

(9540, 3)

Сохраняем csv-файл:

In [ ]:
all_forecasts.to_csv('baseline_forecast.csv', index=False, encoding="utf-8")